In [ ]:
# | default_exp routes.publish

In [ ]:
# | exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

# Publications

In [ ]:
# | export
async def search_publications(
    auth: dmda.DomoAuth,
    search_term: str = None,
    limit=100,
    offset=0,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication/summaries"

    params = {"limit": limit, "offset": offset}

    if search_term:
        params.update({"searchTerm": search_term})

    res = await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        params=params,
        session=session,
        debug_api=debug_api,
    )

    return res

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await search_publications(auth=token_auth)

pd.DataFrame(res.response[0:5])

,id,name,description,dataSets,pages,cards,subscribers,totalSubscribers,invalidSubscribers,invalidDatasetSubscribers,invalidPageSubscribers,invalidCardSubscribers,isV2,hasDuplicate,refreshing,executionType,executionEstimatedCompletionTime,executionStartTime,executionEndTime
0,99a4f772-f0e0-47a6-873a-5870d03b9134,Landing Page QA,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1686863683474,1686863663474,1686863665990
1,e3295d79-4dbe-4994-8f8e-f105b178cc3b,Sandbox_PROD,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1686862478414,1686862458414,1686862460962
2,3ed4e4b3-2d0d-422f-8c2a-e0de94d57ef9,Test OZ,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1687888419847,1687888399847,1687888400466
3,4386c956-205b-4860-a60a-1ba281dc3113,Test OZ,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1688743085044,1688743065044,1688743065624
4,234bff3e-bfe7-4b2d-859e-18077fa6f151,Test OZ,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1687805556975,1687805536975,1687805537496


In [ ]:
# | export
async def get_publication_by_id(
    auth: dmda.DomoAuth,
    publication_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    timeout=10,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication/{publication_id}"

    res = await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        session=session,
        timeout=timeout,
        debug_api=debug_api,
    )

    return res


# generate publish body

In [ ]:
import os
import pandas as pd

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_publication_by_id(
    publication_id=publication_id, auth=token_auth, timeout=10
)

res.response

{'id': '701eb547-4244-43f6-bcda-003ed2c483c7',
 'name': 'test_publish',
 'customerId': 'mmmm-0012-0200',
 'created': 1682580980642,
 'content': {'id': 'dc7b65dd-28fc-4fa6-b469-26873c668549',
  'domain': 'domo-community.domo.com',
  'customerId': 'mmmm-0012-0200',
  'userId': '612085674',
  'type': 'PACKAGE',
  'updated': 1682580980642,
  'hash': 'c85a44e6-d78e-4695-8e3a-f23c22a11ae5',
  'publicationId': '701eb547-4244-43f6-bcda-003ed2c483c7'},
 'description': '',
 'isV2': True,
 'hasDuplicate': False,
 'useDirectContent': True,
 'type': 'CONTENT',
 'isBackup': False,
 'children': [{'id': 'cb257f38-6d6b-46b7-92c5-aee90fde7b66',
   'customerId': '',
   'created': 1692283469815,
   'content': {'id': '891a25be-30b3-4b77-bbc4-18c4619417c4',
    'domain': 'domo-community.domo.com',
    'customerId': '',
    'userId': '1893952720',
    'domoObjectId': '1880852133',
    'type': 'PAGE',
    'updated': 1692283469815,
    'hash': 'fd8d7028-3922-47ac-b89b-12c3ae4d7e45'},
   'isV2': True,
   'hasDu

In [ ]:
# | export
def generate_publish_body(
    url: str,
    sub_domain_ls: [str],
    content_ls: [str],
    name: str,
    description: str,
    unique_id: str,
    is_new: bool,
):
    if not sub_domain_ls:
        sub_domain_ls = []

    if not content_ls:
        content_ls = []

    body = {
        "id": unique_id,
        "name": name,
        "description": description,
        "domain": url,
        "content": content_ls,
        "subscriberDomain": sub_domain_ls,
        "new": str(is_new).lower(),
    }

    return body

In [ ]:
import os
import pandas as pd

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
body = generate_publish_body(
    "domo-community.domo.com", ["test.domo.com"], ["content"], "test", "", "id", True
)
body

{'id': 'id',
 'name': 'test',
 'description': '',
 'domain': 'domo-community.domo.com',
 'content': ['content'],
 'subscriberDomain': ['test.domo.com'],
 'new': 'true'}

In [ ]:
# | export
# Creating publish job for a specific subscriber
async def create_publish_job(
    auth: dmda.DomoAuth,
    body: dict,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication"

    if debug_api:
        print(body)
    res = await gd.get_data(
        auth=auth,
        method="POST",
        url=url,
        body=body,
        session=session,
        debug_api=debug_api,
    )

    return res

In [ ]:
# | export
# Updating existing publish job with content
async def udpate_publish_job(
    auth: dmda.DomoAuth,
    publication_id: str,
    body: dict,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication/{publication_id}"

    res = await gd.get_data(
        auth=auth,
        method="PUT",
        url=url,
        body=body,
        session=session,
        debug_api=debug_api,
    )
    return res


# # finds all jobs waiting to be accepted within the subscriber

# Subscriptions


In [ ]:
# | export
async def get_subscription_summaries(
    auth: dmda.DomoAuth, session: httpx.AsyncClient = None, debug_api: bool = False
) -> rgd.ResponseGetData:
    """retrieves a summary of existing subscriptions"""
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/summaries"

    res = await gd.get_data(
        auth=auth, method="GET", url=url, session=session, debug_api=debug_api
    )
    return res

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_subscription_summaries(auth=token_auth)

pd.DataFrame(res.response[0:5])

""


In [ ]:
# | export
async def get_subscription_invititations(
    auth: dmda.DomoAuth, session: httpx.AsyncClient = None, debug_api: bool = False
) -> rgd.ResponseGetData:
    """retrieves a list of subscription invitations"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/invites"

    res = await gd.get_data(
        auth=auth, method="GET", url=url, session=session, debug_api=debug_api
    )
    return res

#### sample implementation of get_subscription_invitations

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_subscription_invititations(auth=token_auth)

pd.DataFrame(res.response[0:5])

""


In [ ]:
# | export
async def accept_invite_by_id(
    auth: dmda.DomoAuth,
    subscription_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    """this takes get_subscription_invites_list into account and accepts - not instant"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/{subscription_id}"

    res = await gd.get_data(
        auth=auth, method="POST", url=url, session=session, debug_api=debug_api
    )
    return res

In [ ]:
# | export
async def accept_invite_by_id_v2(
    auth: dmda.DomoAuth,
    publication_id: str,
    owner_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    """this takes get_subscription_invites_list into account and accepts - not instant"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/subscriptions/v2"

    body = {
        "publicationId": publication_id,
        "customerId": "",
        "domain": "",
        "groupIds": [],
        "userId": owner_id,
        "userIds": [],
    }

    res = await gd.get_data(
        auth=auth,
        method="POST",
        url=url,
        body=body,
        session=session,
        debug_api=debug_api,
    )
    return res

In [ ]:
# | export
async def refresh_publish_jobs(
    auth: dmda.DomoAuth,
    publish_ids: list,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    """Refreshing list of publish jobs. Typically "instance" = publisher instance"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication/refresh"

    body = {"publicationIds": publish_ids}

    res = await gd.get_data(
        auth=auth,
        method="PUT",
        url=url,
        body=body,
        session=session,
        debug_api=debug_api,
    )
    return res

In [ ]:
# | hide

import nbdev

nbdev.nbdev_export()